# ARTIST ANALYSIS: SUCCESS RATES, TIMELINES, METRICS

In [111]:
import pandas as pd
df_artists = pd.read_csv("artists_df.csv")
df_timeline = pd.read_csv("timeline_df.csv")
df_artists = df_artists.set_index("Unnamed: 0").rename_axis("name", axis = 0)
df_timeline = df_timeline.set_index("Unnamed: 0").rename_axis("release_date", axis = 0)